In [ ]:
# Import dependencies
import pandas as pd
import numpy as np
import json
from datetime import datetime as dt
pd.set_option('max_colwidth', 400)

# Set path
path = "./data/"

## Extract Data from  crowdfunding.xlsx file


In [ ]:
# Read the data into a Pandas DataFrame
crowdfunding_info_df = pd.read_excel(f"{path}crowdfunding.xlsx")
crowdfunding_info_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays


## Split the Category and Subcategory into two Separate Columns

In [ ]:
# Assign the category and subcategory values to category and subcategory columns
crowdfunding_info_df[['category', 'subcategory']] = crowdfunding_info_df["category & sub-category"].str.split('/', expand = True)
crowdfunding_info_df.head()

,cf_id,contact_id,company_name,blurb,goal,pledged,outcome,backers_count,country,currency,launched_at,deadline,staff_pick,spotlight,category & sub-category,category,subcategory
0,147,4661,"Baldwin, Riley and Jackson",Pre-emptive tertiary standardization,100,0,failed,0,CA,CAD,1581573600,1614578400,False,False,food/food trucks,food,food trucks
1,1621,3765,Odom Inc,Managed bottom-line architecture,1400,14560,successful,158,US,USD,1611554400,1621918800,False,True,music/rock,music,rock
2,1812,4187,"Melton, Robinson and Fritz",Function-based leadingedge pricing structure,108400,142523,successful,1425,AU,AUD,1608184800,1640844000,False,False,technology/web,technology,web
3,2156,4941,"Mcdonald, Gonzalez and Ross",Vision-oriented fresh-thinking conglomeration,4200,2477,failed,24,US,USD,1634792400,1642399200,False,False,music/rock,music,rock
4,1365,2199,Larson-Little,Proactive foreground core,7600,5265,failed,53,US,USD,1608530400,1629694800,False,False,theater/plays,theater,plays


## Create Two Separate Data Frames: Category and Subcategory. Export Data Frames as Two CSV Files: category and subcategory.

In [ ]:
# Get the unique categories and subcategories in separate lists
categories = crowdfunding_info_df['category'].unique().tolist()
subcategories = crowdfunding_info_df['subcategory'].unique().tolist()
print(categories)
print(subcategories)

['food', 'music', 'technology', 'theater', 'film & video', 'publishing', 'games', 'photography', 'journalism']
['food trucks', 'rock', 'web', 'plays', 'documentary', 'electric music', 'drama', 'indie rock', 'wearables', 'nonfiction', 'animation', 'video games', 'shorts', 'fiction', 'photography books', 'radio & podcasts', 'metal', 'jazz', 'translations', 'television', 'mobile games', 'world music', 'science fiction', 'audio']


In [ ]:
# Get the number of distinct values in the categories and subcategories lists
cat_total = len(categories)
sub_cat_total = len(subcategories)

# Generate arrays for numeric part of the category and subcatogory ids
category_ids = np.arange(0, cat_total)
subcategory_ids = np.arange(0, sub_cat_total)

# Generate category and subcateogory ids (cat0 and subcat0 .. catn and subcatn)
cat_ids = [f'cat{id}' for id in category_ids]
scat_ids = [f'subcat{id}' for id in subcategory_ids]

print(cat_ids)
print(scat_ids)

['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8']
['subcat0', 'subcat1', 'subcat2', 'subcat3', 'subcat4', 'subcat5', 'subcat6', 'subcat7', 'subcat8', 'subcat9', 'subcat10', 'subcat11', 'subcat12', 'subcat13', 'subcat14', 'subcat15', 'subcat16', 'subcat17', 'subcat18', 'subcat19', 'subcat20', 'subcat21', 'subcat22', 'subcat23']


In [ ]:
# Create a DataFrame with unique category names
category_df = pd.DataFrame({'category_id': cat_ids,
                           'category': categories})
# Create a DataFrame with unique subcategory names
subcategory_df = pd.DataFrame({'subcategory_id': scat_ids,
                           'subcategory': subcategories})

In [ ]:
category_df.sample(9)

,category_id,category
8,cat8,journalism
2,cat2,technology
3,cat3,theater
6,cat6,games
0,cat0,food
7,cat7,photography
1,cat1,music
5,cat5,publishing
4,cat4,film & video


In [ ]:
subcategory_df.sample(9)

,subcategory_id,subcategory
3,subcat3,plays
5,subcat5,electric music
4,subcat4,documentary
1,subcat1,rock
20,subcat20,mobile games
10,subcat10,animation
13,subcat13,fiction
17,subcat17,jazz
9,subcat9,nonfiction


In [ ]:
# Export categories_df and subcategories_df as CSV files.
category_df.to_csv(f"{path}category.csv", index=False)

subcategory_df.to_csv(f"{path}subcategory.csv", index=False)

## Transform Crwodfunding Data to Make it Ready for PostgreSQL

In [ ]:
# Create a copy of the crowdfunding_info_df DataFrame name campaign_df
campaign_df = crowdfunding_info_df.copy()

# Rename the blurb, launched_at, and deadline columns
campaign_df.rename(columns={'blurb':'description', 'launched_at':'launched_date', 'deadline':'end_date'}, inplace=True)

# Convert the goal and pledged columns to a float data type.
campaign_df = campaign_df.astype({'goal':'float','pledged':'float'})

# Format the launched_date and end_date columns to datetime format
campaign_df['launched_date'] = pd.to_datetime(campaign_df['launched_date'],unit='s').dt.date
campaign_df['end_date'] = pd.to_datetime(campaign_df['end_date'],unit='s').dt.date

campaign_df.sample()

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory
841,55,2940,"Garcia, Dunn and Richardson",Automated even-keeled emulation,9100.0,12991.0,successful,155,US,USD,2020-10-11,2021-03-10,False,False,technology/web,technology,web


In [ ]:
# Check the datatypes
campaign_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 17 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   cf_id                    1000 non-null   int64  
 1   contact_id               1000 non-null   int64  
 2   company_name             1000 non-null   object 
 3   description              1000 non-null   object 
 4   goal                     1000 non-null   float64
 5   pledged                  1000 non-null   float64
 6   outcome                  1000 non-null   object 
 7   backers_count            1000 non-null   int64  
 8   country                  1000 non-null   object 
 9   currency                 1000 non-null   object 
 10  launched_date            1000 non-null   object 
 11  end_date                 1000 non-null   object 
 12  staff_pick               1000 non-null   bool   
 13  spotlight                1000 non-null   bool   
 14  category & sub-category  

In [ ]:
# Merge the campaign_df with the category_df on the "category" column and
# the subcategory_df on the "subcategory" to get category and subcatogory ids
campaign_merged_df = campaign_df.merge(category_df, on='category')
campaign_merged_df = campaign_merged_df.merge(subcategory_df, on='subcategory')
campaign_merged_df.tail(10)

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,staff_pick,spotlight,category & sub-category,category,subcategory,category_id,subcategory_id
990,399,5983,"Dixon, Perez and Banks",Re-engineered encompassing definition,8300.0,2111.0,failed,57,CA,CAD,2021-12-29,2022-01-16,False,False,photography/photography books,photography,photography books,cat7,subcat14
991,1378,2873,Clements Ltd,Persistent bandwidth-monitored framework,3700.0,1343.0,failed,52,US,USD,2020-07-07,2021-07-27,False,False,photography/photography books,photography,photography books,cat7,subcat14
992,806,2961,Walter Inc,Streamlined 5thgeneration intranet,10000.0,8142.0,failed,263,AU,AUD,2020-12-20,2021-01-31,False,False,photography/photography books,photography,photography books,cat7,subcat14
993,1563,3043,"Sanders, Farley and Huffman",Cross-group clear-thinking task-force,172000.0,55805.0,failed,1691,US,USD,2021-04-08,2021-12-18,True,False,photography/photography books,photography,photography books,cat7,subcat14
994,1951,6013,Rodriguez-Robinson,Ergonomic methodical hub,5900.0,4997.0,failed,114,IT,EUR,2021-07-23,2021-11-25,False,True,photography/photography books,photography,photography books,cat7,subcat14
995,461,3521,Erickson-Rogers,De-engineered even-keeled definition,9800.0,7608.0,canceled,75,IT,EUR,2021-07-03,2021-07-08,False,True,photography/photography books,photography,photography books,cat7,subcat14
996,1095,3565,Young and Sons,Innovative disintermediate encryption,2400.0,4596.0,successful,144,US,USD,2020-11-28,2021-02-28,False,False,journalism/audio,journalism,audio,cat8,subcat23
997,1388,4517,Valenzuela-Cook,Total incremental productivity,6700.0,7496.0,successful,300,US,USD,2021-01-04,2021-10-04,False,False,journalism/audio,journalism,audio,cat8,subcat23
998,2448,2389,Little Ltd,Decentralized bandwidth-monitored ability,6900.0,12155.0,successful,419,US,USD,2021-01-25,2021-04-18,False,False,journalism/audio,journalism,audio,cat8,subcat23
999,602,2153,Giles-Smith,Right-sized web-enabled intranet,9700.0,11929.0,successful,331,US,USD,2022-02-01,2022-02-16,False,False,journalism/audio,journalism,audio,cat8,subcat23


In [ ]:
# Drop unwanted columns
campaign_cleaned = campaign_merged_df.drop(columns = ["staff_pick", "spotlight", "category & sub-category", "category", "subcategory"])

campaign_cleaned.sample(10)

,cf_id,contact_id,company_name,description,goal,pledged,outcome,backers_count,country,currency,launched_date,end_date,category_id,subcategory_id
356,2957,6070,"Tucker, Schmidt and Reid",Multi-layered encompassing installation,3500.0,6527.0,successful,86,US,USD,2021-05-07,2021-12-28,cat3,subcat3
923,3208,5165,Cuevas-Morales,Public-key coherent ability,900.0,8703.0,successful,86,DK,DKK,2020-11-02,2021-04-07,cat6,subcat11
748,526,1312,Brooks-Rodriguez,Implemented intangible instruction set,5100.0,5421.0,successful,164,US,USD,2021-02-19,2021-04-09,cat4,subcat6
591,23,3790,Webb Group,Public-key actuating projection,108700.0,87293.0,failed,831,US,USD,2020-12-18,2021-03-14,cat3,subcat3
891,387,3193,Gonzalez-Robbins,Up-sized responsive protocol,4700.0,8829.0,successful,80,US,USD,2021-04-09,2022-02-18,cat5,subcat18
558,1494,3801,Bradford-Silva,Reverse-engineered composite hierarchy,180400.0,115396.0,failed,1748,US,USD,2020-10-09,2021-06-26,cat3,subcat3
208,2371,2996,Stevens Inc,Adaptive 24hour projection,8200.0,1546.0,canceled,37,US,USD,2021-10-08,2021-11-27,cat1,subcat17
43,1432,2712,Johnson Group,Vision-oriented interactive solution,7000.0,5177.0,failed,67,US,USD,2021-04-22,2022-01-08,cat0,subcat0
718,1521,4997,Freeman-French,Multi-layered optimal application,7200.0,6115.0,failed,75,US,USD,2020-02-21,2021-02-23,cat4,subcat4
510,1354,2350,Joseph LLC,Decentralized logistical collaboration,94900.0,194166.0,successful,3596,US,USD,2021-01-20,2021-05-11,cat3,subcat3


In [ ]:
# Export the DataFrame as a CSV file.
campaign_cleaned.to_csv(f"{path}campaign.csv", index=False)

## Extract and Transform Contacts Data

In [ ]:
# Read the data into a Pandas DataFrame. Use the header=2 parameter when reading in the data
contact_info_df = pd.read_excel(f"{path}contacts.xlsx", header=2)
contact_info_df.head()

,
0,contact_info
1,"{""contact_id"": 4661, ""name"": ""Cecilia Velasco"", ""email"": ""cecilia.velasco@rodrigues.fr""}"
2,"{""contact_id"": 3765, ""name"": ""Mariana Ellis"", ""email"": ""mariana.ellis@rossi.org""}"
3,"{""contact_id"": 4187, ""name"": ""Sofie Woods"", ""email"": ""sofie.woods@riviere.com""}"
4,"{""contact_id"": 4941, ""name"": ""Jeanette Iannotti"", ""email"": ""jeanette.iannotti@yahoo.com""}"


In [ ]:
# Get the values from contact_info column
contacts_list = contact_info_df.loc[1:]
# Flatten the ndarray
contact_strings = contacts_list.loc[1:].values.reshape(len(contacts_list))
# Convert strings to dictionary
contacts = [json.loads(s) for s in contact_strings]
# Create a df
contacts_df = pd.DataFrame.from_dict(contacts)

contacts_df.head()

,contact_id,name,email
0,4661,Cecilia Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana Ellis,mariana.ellis@rossi.org
2,4187,Sofie Woods,sofie.woods@riviere.com
3,4941,Jeanette Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel Sorgatz,samuel.sorgatz@gmail.com


In [ ]:
# Split name into first and last and add respective columns

contacts_df[["first_name", "last_name"]] = contacts_df.name.str.split(' ', expand=True)
contacts_df.drop(columns=["name"], inplace=True)
# Rearrange columns
contacts_df = contacts_df[["contact_id", "first_name", "last_name", "email"]]

contacts_df.head()

,contact_id,first_name,last_name,email
0,4661,Cecilia,Velasco,cecilia.velasco@rodrigues.fr
1,3765,Mariana,Ellis,mariana.ellis@rossi.org
2,4187,Sofie,Woods,sofie.woods@riviere.com
3,4941,Jeanette,Iannotti,jeanette.iannotti@yahoo.com
4,2199,Samuel,Sorgatz,samuel.sorgatz@gmail.com


In [ ]:
# Export the DataFrame as a CSV file.
contacts_df.to_csv(f"{path}contacts.csv", encoding='utf8', index=False)